# Fetch and Write MarketStore

In [ ]:
import datetime
import ccxt
import numpy as np
import pandas as pd
import pymarketstore as pymkts
import time

# Required parameters
binanceApiKey = 'your binance API key'
binanceSecret = 'your binance secret'

endpoint = "http://localhost:5993/rpc"  # Your MarketStore Url

symbol = 'BTC/USDT'
tbk = f'Binance-{symbol.replace("/", "-")}-Custom/1Min/OHLCV-4H'  # MarketStore tbk 

# Optional parameters
limit = 400  # The amount of data acquired at one time

# init binance with ccxt
exchange = ccxt.binance()
exchange.apiKey = binanceApiKey
exchange.secret = binanceSecret

# set end time
end_time = datetime.datetime.now()

print('------------------ Starting ------------------')

# same data flag
flag = None

# loop data by limit
while True:
    # Calculate start time
    start_time = end_time - datetime.timedelta(hours=limit * 4)

    # Convert time to millisecond timestamp
    start_timestamp = int(start_time.timestamp() * 1000)
    end_timestamp = int(end_time.timestamp() * 1000)

    # Fetch historical data
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe='4h', since=start_timestamp, limit=limit)
    except Exception as e:
        print("Error fetching ohlcv.Try again in 1 second.",e)
        time.sleep(1)
        continue

    # Convert timestamp format
    for item in ohlcv:
        item[0] = pd.to_datetime(item[0], unit='ms').timestamp()

    # Format the data from [[1701820800000, 44131.1, 44981.47, 20000.0, 43769.54, 200.52232],[1701907200000, 43770.54, 45649.78, 1000.0, 43270.1, 431.88535]] 
    # replaced by [ (1701820800000, 44131.1, 44981.47, 20000.0, 43769.54, 200.52232),(1701907200000, 43770.54, 45649.78, 1000.0, 43270.1, 431.88535)]
    converted_data = [tuple(item) for item in ohlcv]

    # Set dType
    dType = [('Epoch', 'i8'), ('open', 'f4'), ('high', 'f4'), ('low', 'f4'), ('close', 'f4'), ('volume', 'f4')]

    # Convert NumPy
    result = np.array(converted_data, dtype=dType)

    # Set MarketStore Url
    cli = pymkts.Client(endpoint=endpoint)

    # Print write result
    response = cli.write(result, tbk)
    if response['responses'] is not None:
        print("Write Error:", response['responses'])
    else:
        # Update end time
        end_time = start_time
        print(f'Write Success: {len(result)} rows to {tbk}.')

    if flag == ohlcv[0][0]:
        print('------------------ Finished ------------------')
        break

    flag = ohlcv[0][0]
    print(ohlcv[0][0])

# Query From MarketStore

In [ ]:
import pymarketstore as pymkts

endpoint = "http://localhost:5993/rpc"  # Your MarketStore Url

symbol = 'BTC/USDT'
pymkts.Client(endpoint=endpoint).query(
    pymkts.Params(symbols=f'Binance-{symbol.replace("/", "-")}-Custom', timeframe="1Min",
                  attrgroup="OHLCV-4H")).first().df()